https://towardsdatascience.com/image-captioning-using-deep-learning-fe0d929cf337




In [21]:
!pip install graphviz 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
import cv2
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Input, Embedding, Conv2D, Concatenate, Flatten, Add, Dropout, GRU
import random
import datetime
from nltk.translate.bleu_score import sentence_bleu

In [2]:
def get_random_set():
    obj = reqd[np.random.randint(len(reqd))]
    for i in range(len(obj["imgs"])):
        plt.subplot(1, len(obj["imgs"]),i+1)
        plt.axis('off')
        img = plt.imread(get_img_path(obj["imgs"][i]))
        plt.imshow(img)
    print(obj["desc"])

def get_random_set2(df, k):
    obj = df[k]
    for i in range(len(obj["imgs"])):
        plt.subplot(1, len(obj["imgs"]),i+1)
        plt.axis('off')
        img = plt.imread(get_img_path(obj["imgs"][i]))
        plt.imshow(img)
    print(obj["desc"])
    
def get_img_path(x):
    return "./imgs/"+x+".png"

from PIL import Image
def load_image(img_name):
    '''Function to load the image'''
    image = Image.open(img_name)
    image = image.resize((224,224))
    image_array = np.asarray(image.convert("RGB"))
    image_array = image_array / 255.
    
    X = np.expand_dims(image_array, axis=0)
    X = np.asarray(X) 
    return X

In [3]:
train_dataset = pd.read_csv('df_train.csv')
test_dataset = pd.read_csv('df_test.csv')
cv_dataset = pd.read_csv('df_cv.csv')

In [4]:
train_dataset

,Unnamed: 0,desc,img1,img2,img3,img4,img5
0,0,startseq right upper lobe airspace disease con...,CXR3069_IM-1432-1001,CXR3069_IM-1432-2001,NaN,NaN,NaN
1,1,startseq mediastinal contours are normal lungs...,CXR2629_IM-1116-1001,CXR2629_IM-1116-2001,NaN,NaN,NaN
2,2,startseq cardiomediastinal contours are unchan...,CXR1888_IM-0576-1001,CXR1888_IM-0576-4004,NaN,NaN,NaN
3,3,startseq heart size normal the lungs are clear...,CXR2248_IM-0844-1001,CXR2248_IM-0844-1002,NaN,NaN,NaN
4,4,startseq the cardiomediastinal contours are wi...,CXR3408_IM-1648-1001,CXR3408_IM-1648-1002,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2063,2331,startseq the heart normal size the mediastinum...,CXR2243_IM-0840-1001,CXR2243_IM-0840-2001,CXR2243_IM-0840-3001,CXR2243_IM-0840-4001,NaN
2064,2332,startseq normal heart size normal mediastinal ...,CXR1356_IM-0231-1001,CXR1356_IM-0231-2001,NaN,NaN,NaN
2065,2333,startseq frontal and lateral views the chest s...,CXR1883_IM-0572-1001,CXR1883_IM-0572-2001,NaN,NaN,NaN
2066,2334,startseq normal heart size and mediastinal con...,CXR2622_IM-1110-1001,CXR2622_IM-1110-1002,NaN,NaN,NaN


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [6]:
gpus

[]

In [7]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(train_dataset.desc)

In [8]:
tokenizer.word_index

{'the': 1,
 'no': 2,
 'are': 3,
 'normal': 4,
 'startseq': 5,
 'endseq': 6,
 'and': 7,
 'pleural': 8,
 'pneumothorax': 9,
 'there': 10,
 'effusion': 11,
 'heart': 12,
 'lungs': 13,
 'size': 14,
 'focal': 15,
 'clear': 16,
 'within': 17,
 'limits': 18,
 'pulmonary': 19,
 'consolidation': 20,
 'silhouette': 21,
 'right': 22,
 'mediastinal': 23,
 'cardiomediastinal': 24,
 'airspace': 25,
 'left': 26,
 'acute': 27,
 'lung': 28,
 'spine': 29,
 'with': 30,
 'changes': 31,
 'disease': 32,
 'unremarkable': 33,
 'structures': 34,
 'stable': 35,
 'mediastinum': 36,
 'bony': 37,
 'contours': 38,
 'contour': 39,
 'thoracic': 40,
 'degenerative': 41,
 'large': 42,
 'without': 43,
 'calcified': 44,
 'mild': 45,
 'seen': 46,
 'effusions': 47,
 'cardiac': 48,
 'appear': 49,
 'osseous': 50,
 'vascularity': 51,
 'opacity': 52,
 'for': 53,
 'abnormality': 54,
 'intact': 55,
 'evidence': 56,
 'visualized': 57,
 'upper': 58,
 'opacities': 59,
 'vasculature': 60,
 'lateral': 61,
 'noted': 62,
 'lobe': 63,
 

In [ ]:
# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
vocab_size = len(tokenizer.word_counts) + 1
vocab_size

1313

In [131]:
f = open('Image_features_attention.pickle','rb') # contains the features from chexNet
Xnet_Features = pickle.load(f)
f.close()

In [10]:
try:
    with open(r"embedding_matrix.pickle", "rb") as output_file:
        embedding_matrix = pickle.load(output_file)
    print("loaded")
except:
    embedding_matrix = np.zeros((vocab_size,300))
    filepath = r"./utils/glove.6B/glove.6B.300d.txt"
    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in tokenizer.word_index.keys():
                idx = tokenizer.word_index[word]
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:300]
    with open(r"embedding_matrix.pickle", "wb") as output_file:
        pickle.dump(embedding_matrix, output_file)

loaded


In [152]:
embedding_matrix[1].shape

(300,)

In [11]:
BATCH_SIZE = 14

In [59]:
def load_image(id_, report):
    '''Loads the Image Features with their corresponding Ids'''
#     test_2827 train_2335
    if id_ > 2827:
        img_feature = Xnet_Features["cv_"+str(id_)][0]
    elif id_ > 2335:
        img_feature = Xnet_Features["test_"+str(id_)][0]
    else:
        img_feature = Xnet_Features["train_"+str(id_)][0]
    return img_feature, report

In [60]:
def create_dataset(img_name, caption):
    dataset = tf.data.Dataset.from_tensor_slices((img_name, caption))
    # Use map to load the numpy files in parallel
    dataset = dataset.map(lambda item1, item2: tf.numpy_function(load_image, [item1, item2], [tf.float32, tf.string]),
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # Shuffle and batch
    dataset = dataset.shuffle(500).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

In [61]:
train_generator = create_dataset(train_dataset.iloc[:,0], train_dataset.desc)
train_generator

<PrefetchDataset shapes: (<unknown>, <unknown>), types: (tf.float32, tf.string)>

In [80]:
cv_generator = create_dataset(cv_dataset.iloc[:,0], cv_dataset.desc)

In [147]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.04656   ,  0.21318001, -0.0074364 , ...,  0.0090611 ,
        -0.20988999,  0.053913  ],
       [-0.16843   , -0.037651  , -0.17304   , ..., -0.49983999,
        -0.17079   ,  0.46430999],
       ...,
       [-0.33412001,  0.018642  , -0.01061   , ...,  0.96328998,
         0.20008001, -0.1167    ],
       [-0.0085603 ,  0.094092  ,  0.33443999, ..., -0.41503999,
        -0.55855   ,  0.12241   ],
       [-0.054388  ,  0.086841  ,  0.10388   , ..., -0.31617001,
        -0.14820001, -0.072753  ]])

In [63]:
input1 = Input(shape=(2048), name='Image_1')
dense1 = Dense(256, kernel_initializer=tf.keras.initializers.glorot_uniform(seed = 56), name='dense_encoder')(input1)

input2 = Input(shape=(153), name='Text_Input')
emb_layer = Embedding(input_dim = vocab_size, output_dim = 300, input_length=153, mask_zero=True, trainable=False, 
                weights=[embedding_matrix], name="Embedding_layer")
emb = emb_layer(input2)

LSTM1 = LSTM(units=256, activation='tanh', recurrent_activation='sigmoid', use_bias=True, 
            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=23),
            recurrent_initializer=tf.keras.initializers.orthogonal(seed=7),
            bias_initializer=tf.keras.initializers.zeros(), return_sequences=True, name="LSTM1")(emb)
#LSTM1_output = LSTM1(emb)

LSTM2 = LSTM(units=256, activation='tanh', recurrent_activation='sigmoid', use_bias=True, 
            kernel_initializer=tf.keras.initializers.glorot_uniform(seed=23),
            recurrent_initializer=tf.keras.initializers.orthogonal(seed=7),
            bias_initializer=tf.keras.initializers.zeros(), name="LSTM2")
LSTM2_output = LSTM2(LSTM1)

dropout1 = Dropout(0.5, name='dropout1')(LSTM2_output)

dec =  tf.keras.layers.Add()([dense1, dropout1])

fc1 = Dense(256, activation='relu', kernel_initializer=tf.keras.initializers.he_normal(seed = 63), name='fc1')
fc1_output = fc1(dec)
dropout2 = Dropout(0.4, name='dropout2')(fc1_output)
output_layer = Dense(vocab_size, activation='softmax', name='Output_layer')
output = output_layer(dropout2)

encoder_decoder = Model(inputs = [input1, input2], outputs = output)
encoder_decoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Text_Input (InputLayer)         [(None, 153)]        0                                            
__________________________________________________________________________________________________
Embedding_layer (Embedding)     (None, 153, 300)     393900      Text_Input[0][0]                 
__________________________________________________________________________________________________
LSTM1 (LSTM)                    (None, 153, 256)     570368      Embedding_layer[0][0]            
__________________________________________________________________________________________________
Image_1 (InputLayer)            [(None, 2048)]       0                                            
____________________________________________________________________________________________

In [64]:
tf.keras.utils.plot_model(encoder_decoder)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [65]:
optimizer = tf.keras.optimizers.Adam(0.001)

In [66]:
loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=False, reduction='auto')

def maskedLoss(y_true, y_pred):
    #getting mask value
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))
    
    #calculating the loss
    loss_ = loss_function(y_true, y_pred)
    
    #converting mask dtype to loss_ dtype
    mask = tf.cast(mask, dtype=loss_.dtype)
    
    #applying the mask to loss
    loss_ = loss_*mask
    
    #getting mean over all the values
    loss_ = tf.reduce_mean(loss_)
    return loss_

In [67]:
embedding_matrix.shape

(1313, 300)

In [68]:
encoder_decoder.compile(optimizer, loss = maskedLoss)

In [69]:
def bytes_to_string(arr):
    '''The generator gives provides data in bytes. This function converts them back to strings for manipulation'''
    for i in range(len(arr)):
        arr[i] = arr[i].decode('utf-8')
    return arr

In [70]:
def convert(images, reports):
    '''This function takes the batch of data and converts them into a new dataset(A WORD BY WORD DATASET)'''
    imgs = []
    in_reports = []
    out_reports = []
    for i in range(len(images)):
        sequence = [tokenizer.word_index[e] for e in reports[i].split() if e in tokenizer.word_index.keys()]
      #  print(sequence)
        for j in range(1,len(sequence)):
            in_seq = sequence[:j]
            out_seq = sequence[j]
            out_seq = tf.keras.utils.to_categorical(out_seq, num_classes=vocab_size)
            imgs.append(images[i])
          #  print(in_seq)
            in_reports.append(in_seq)
           # print(out_seq)
            out_reports.append(out_seq)
    return np.array(imgs), np.array(in_reports), np.array(out_reports)

In [71]:
EPOCH = 20

In [72]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'Tensorboard/logs_m1/fit3/' + current_time + '/train'
val_log_dir = 'Tensorboard/logs_m1/fit3/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
val_summary_writer = tf.summary.create_file_writer(val_log_dir)

In [160]:
for img, report in train_generator:
        r1 = bytes_to_string(report.numpy())
        img_input, rep_input, output_word = convert(img.numpy(), r1)
#         print(rep_input)
#         print(rep_input)
        rep_input = pad_sequences(rep_input, maxlen=153, padding='post')
#         print(rep_input)
        results = encoder_decoder.train_on_batch([img_input, rep_input], output_word)
        print(results)
        break

C:\Users\prana\AppData\Local\Temp\ipykernel_28904\85539252.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(imgs), np.array(in_reports), np.array(out_reports)


0.00125814788043499


In [74]:
epoch_train_loss = []
epoch_val_loss = []

for epoch in range(20):
    print('EPOCH : ',epoch+1)
    start = time.time()
    batch_loss_tr = 0
    batch_loss_vl = 0
    
    for img, report in train_generator:
        r1 = bytes_to_string(report.numpy())
        img_input, rep_input, output_word = convert(img.numpy(), r1)
        rep_input = pad_sequences(rep_input, maxlen=153, padding='post')
        results = encoder_decoder.train_on_batch([img_input, rep_input], output_word)

        batch_loss_tr += results
    
    train_loss = batch_loss_tr/(len(train_dataset)//14)
 #   print('Saving Tensorboard')
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss, step = epoch)
    
    for img, report in cv_generator:
        
        r1 = bytes_to_string(report.numpy())
        img_input, rep_input, output_word = convert(img.numpy(), r1)
        rep_input = pad_sequences(rep_input, maxlen=153, padding='post')
        results = encoder_decoder.test_on_batch([img_input, rep_input], output_word)
        batch_loss_vl += results
    
    val_loss = batch_loss_vl/(len(cv_dataset)//14)
    
    with val_summary_writer.as_default():
        tf.summary.scalar('loss', val_loss, step = epoch)

    epoch_train_loss.append(train_loss)

    epoch_val_loss.append(val_loss)
    
    print('Training Loss: {},  Val Loss: {}'.format(train_loss, val_loss))
    print('Time Taken for this Epoch : {} sec'.format(time.time()-start))   
    encoder_decoder.save_weights('Weights_re/encoder_decoder_epoch_'+ str(epoch+1) + '.h5')

EPOCH :  1


C:\Users\prana\AppData\Local\Temp\ipykernel_28904\85539252.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(imgs), np.array(in_reports), np.array(out_reports)


Training Loss: 0.0033988541603519194,  Val Loss: 0.0030873050132105427
Time Taken for this Epoch : 392.8474745750427 sec
EPOCH :  2
Training Loss: 0.0029646353669413902,  Val Loss: 0.002589465021305988
Time Taken for this Epoch : 413.4568979740143 sec
EPOCH :  3
Training Loss: 0.0025642738902472515,  Val Loss: 0.0022834883176631503
Time Taken for this Epoch : 339.41609168052673 sec
EPOCH :  4
Training Loss: 0.0022942615596919643,  Val Loss: 0.0020640030570869003
Time Taken for this Epoch : 331.6139030456543 sec
EPOCH :  5
Training Loss: 0.0021123053081536372,  Val Loss: 0.0019294517415185128
Time Taken for this Epoch : 330.2767524719238 sec
EPOCH :  6
Training Loss: 0.0019868127241426583,  Val Loss: 0.0018830453045666218
Time Taken for this Epoch : 331.7625858783722 sec
EPOCH :  7
Training Loss: 0.0018867052644871327,  Val Loss: 0.001786282627783235
Time Taken for this Epoch : 3036.2138550281525 sec
EPOCH :  8
Training Loss: 0.0018009650980306117,  Val Loss: 0.001722438294710892
Time T

In [76]:
# encoder
encoder_input = encoder_decoder.input[0]
encoder_output = encoder_decoder.get_layer('dense_encoder').output
encoder_model = Model(encoder_input, encoder_output)


# decoder
text_input = encoder_decoder.input[1]
enc_output = Input(shape=(256,), name='Enc_Output')
text_output = encoder_decoder.get_layer('LSTM2').output
add1 = tf.keras.layers.Add()([text_output, enc_output])
fc_1 = fc1(add1)
decoder_output = output_layer(fc_1)
decoder_model = Model(inputs = [text_input, enc_output], outputs = decoder_output)

In [81]:
encoder_decoder.input

[<KerasTensor: shape=(None, 2048) dtype=float32 (created by layer 'Image_1')>,
 <KerasTensor: shape=(None, 153) dtype=float32 (created by layer 'Text_Input')>]

In [92]:
def greedysearch(img):
    image = Xnet_Features[img] # Extract the initial chexnet features for the images
    input_ = 'startseq'  # initial partial report
    image_features = encoder_model.predict(image) # encoder output
    
    result = [] 
    for i in range(153):
        input_tok = [tokenizer.word_index[w] for w in input_.split()]
        input_padded = pad_sequences([input_tok], 153, padding='post')
        predictions = decoder_model.predict([input_padded, image_features])
        arg = np.argmax(predictions)
        if arg != tokenizer.word_index['endseq']:   # endseq
            result.append(tokenizer.index_word[arg])
            input_ = input_ + ' ' + tokenizer.index_word[arg]
        else:
            break
    rep = ' '.join(e for e in result)
    return rep

In [121]:
for i in range(18):
    res36 = greedysearch("train_"+str(i)) # pred
    print("Predicted : "+res36)
    print("Actual : "+train_dataset.iloc[i].desc)
    print()

Predicted : the heart normal size the mediastinum unremarkable the lungs are clear
Actual : startseq right upper lobe airspace disease consistent with pneumonia given patient history the lungs are otherwise clear no pleural effusions pneumothoraces heart and mediastinum normal size and contour endseq

Predicted : the heart normal size the mediastinum unremarkable the lungs are clear
Actual : startseq mediastinal contours are normal lungs are clear there no pneumothorax large pleural effusion endseq

Predicted : the heart normal size the mediastinum unremarkable the lungs are clear
Actual : startseq cardiomediastinal contours are unchanged there are stable fractures several lungs are hyperexpanded but clear no pneumothorax pleural effusion degenerative changes are seen the spine endseq

Predicted : the lungs are clear there no pleural effusion the heart and mediastinum are normal the skeletal structures and soft tissues are normal
Actual : startseq heart size normal the lungs are clear 

In [118]:
train_dataset.iloc[i].desc #actual

'startseq the cardiac contours are normal the lungs are clear thoracic spondylosis endseq'

In [122]:
def beamsearch(image, beam_width):
    
    start = [tokenizer.word_index['startseq']]

    sequences = [[start, 0]]
    
    img_features = Xnet_Features[image]
    img_features = encoder_model.predict(img_features)
    finished_seq = []
    
    for i in range(153):
        all_candidates = []
        new_seq = []
        for s in sequences:

            text_input = pad_sequences([s[0]], 153, padding='post')
            predictions = decoder_model.predict([text_input,img_features])
            top_words = np.argsort(predictions[0])[-beam_width:]
            seq, score = s
            
            for t in top_words:
                candidates = [seq + [t], score - np.log(predictions[0][t])]
                all_candidates.append(candidates)
                
        sequences = sorted(all_candidates, key = lambda l: l[1])[:beam_width]
        # checks for 'endseq' in each seq in the beam
        count = 0
        for seq,score in sequences:
            if seq[len(seq)-1] == tokenizer.word_index['endseq']:
                score = score/len(seq)   # normalized
                finished_seq.append([seq, score])
                count+=1
            else:
                new_seq.append([seq, score])
        beam_width -= count
        sequences = new_seq
        
        # if all the sequences reaches its end before 155 timesteps
        if not sequences:
            break
        else:
            continue
        
    sequences = finished_seq[-1] 
    rep = sequences[0]
    score = sequences[1]
    temp = []
    rep.pop(0)
    for word in rep:
        if word != tokenizer.word_index['endseq']:
            temp.append(tokenizer.index_word[word])
        else:
            break    
    rep = ' '.join(e for e in temp)        
    
    return rep, score

In [126]:
res0 = beamsearch("train_1", 5) # beam_width = 5 
res0

('the cardiomediastinal silhouette within normal limits for appearance no focal areas pulmonary consolidation no pneumothorax no pleural effusion the thoracic spine appears intact',
 0.27516274766268906)

In [125]:
train_dataset.iloc[1].desc #actual

'startseq mediastinal contours are normal lungs are clear there no pneumothorax large pleural effusion endseq'

In [128]:
for i in range(18):
    res36 = beamsearch("train_"+str(i), 5)
    print("Predicted : "+res36[0])
    print("Actual : "+train_dataset.iloc[i].desc)
    print()

Predicted : the lungs are clear there no pleural effusion the heart and mediastinum are normal the skeletal structures and soft tissues are normal
Actual : startseq right upper lobe airspace disease consistent with pneumonia given patient history the lungs are otherwise clear no pleural effusions pneumothoraces heart and mediastinum normal size and contour endseq

Predicted : the cardiomediastinal silhouette within normal limits for appearance no focal areas pulmonary consolidation no pneumothorax no pleural effusion the thoracic spine appears intact
Actual : startseq mediastinal contours are normal lungs are clear there no pneumothorax large pleural effusion endseq

Predicted : the cardiomediastinal silhouette within normal limits for size and contour the lungs are normally inflated without evidence focal airspace disease pleural effusion pneumothorax no acute bone abnormality
Actual : startseq cardiomediastinal contours are unchanged there are stable fractures several lungs are hyper

In [129]:
Xnet_Features.keys()

dict_keys(['train_0', 'train_1', 'train_2', 'train_3', 'train_4', 'train_5', 'train_6', 'train_7', 'train_8', 'train_9', 'train_10', 'train_11', 'train_12', 'train_13', 'train_14', 'train_15', 'train_16', 'train_17', 'train_18', 'train_19', 'train_21', 'train_23', 'train_25', 'train_26', 'train_27', 'train_28', 'train_29', 'train_30', 'train_31', 'train_33', 'train_34', 'train_35', 'train_36', 'train_37', 'train_39', 'train_40', 'train_41', 'train_42', 'train_43', 'train_44', 'train_45', 'train_47', 'train_48', 'train_49', 'train_50', 'train_52', 'train_53', 'train_54', 'train_55', 'train_56', 'train_57', 'train_58', 'train_59', 'train_60', 'train_61', 'train_62', 'train_63', 'train_64', 'train_65', 'train_66', 'train_67', 'train_68', 'train_70', 'train_71', 'train_72', 'train_73', 'train_74', 'train_75', 'train_76', 'train_77', 'train_78', 'train_79', 'train_80', 'train_81', 'train_82', 'train_83', 'train_86', 'train_87', 'train_88', 'train_89', 'train_90', 'train_91', 'train_92', 'tr

In [132]:
for i in range(18):
    res36 = beamsearch("test_"+str(i), 5)
    print("Predicted : "+res36[0])
    print("Actual : "+test_dataset.iloc[i].desc)
    print()

Predicted : the cardiomediastinal silhouette within normal limits for appearance no focal areas pulmonary consolidation no pneumothorax no pleural effusion the thoracic spine appears intact
Actual : startseq the heart normal size the pulmonary vascularity within normal limits appearance no focal air space opacities no pleural effusions pneumothorax no acute bony abnormalities endseq

Predicted : the cardiomediastinal silhouette within normal limits for size and contour the lungs are normally inflated without evidence focal airspace disease pleural effusion pneumothorax no acute osseus abnormality
Actual : startseq lucency crosses the left posterior rib visualized portions the thoracic spine are unremarkable mediastinal contours are normal lungs are clear there no pneumothorax large pleural effusion endseq

Predicted : the cardiomediastinal silhouette within normal limits for appearance no focal areas pulmonary consolidation no pneumothorax no pleural effusion the thoracic spine appears

In [134]:
for i in range(18):
    res36 = beamsearch("cv_"+str(i), 5)
    print("Predicted : "+res36[0])
    print("Actual : "+cv_dataset.iloc[i].desc)
    print()

Predicted : the cardiomediastinal silhouette within normal limits for size and contour the lungs are normally inflated without evidence focal airspace disease pleural effusion pneumothorax osseous structures are within normal limits for patient age
Actual : startseq heart size enlarged but stable stable sequela prior granulomatous disease stable sternotomy with fracture the superior most sternotomy the lungs are normally inflated without evidence focal airspace disease pleural effusion pneumothorax degenerative endplate changes the spine endseq

Predicted : the cardiomediastinal silhouette within normal limits for size and contour the lungs are normally inflated without evidence focal airspace disease pleural effusion pneumothorax no acute bone abnormality
Actual : startseq the examination consists frontal and lateral radiographs the chest sternotomy and surgical clips are again seen the cardiomediastinal contours are unchanged there background marked centrilobular emphysema streaky op